In [40]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
GOOGLE_SEARCH_API_KEY=os.getenv("GOOGLE_SEARCH_API_KEY")
GOOGLE_CSE_ID=os.getenv("GOOGLE_CSE_ID")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")

In [42]:
#loading local model
MODEL="deepseek-r1:1.5b"
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
llm = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [43]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = llm | parser 

In [44]:
print(chain.invoke("YO, tell me a joke"))

<think>

</think>

Sure! Here's a light-hearted joke for you:

Why don’t skeletons fight each other?  
Because they don’t have the *gym*! 😄


### step 1. Make a vector store ###

In [45]:
#making a retreiver with vector store out of a pdf document
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path="thesis.pdf")
pages = loader.load_and_split()
print(f"Loaded pdf with {len(pages)} pages")

Loaded pdf with 65 pages


#### Option A : Simple DocArrayInMemorySearch ####

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

#### Option B : Facebook AI Similarity Search (FAISS) ####

In [51]:
#ONLY do this once
from langchain_community.vectorstores import FAISS

#vectorstore = FAISS.from_documents(documents = pages, embedding = embeddings)
#vectorstore.save_local(folder_path='data/',index_name="thesis-index")

In [53]:
vectorstore = FAISS.load_local(folder_path='data/',
                               index_name='thesis-index',
                               embeddings=embeddings,
                               allow_dangerous_deserialization = True
                               )
retriever = vectorstore.as_retriever()

In [54]:
retriever.invoke("reinforcement learning")

[Document(id='3ee8b4a9-b3cf-4e19-b88a-48001ca1dd55', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-08T14:14:32+00:00', 'author': '', 'keywords': '', 'moddate': '2025-08-08T14:14:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'thesis.pdf', 'total_pages': 65, 'page': 20, 'page_label': '14'}, page_content='3 Methodology\na custom dataset within the Blocksworld domain. Usually, problem statements from\nBlocksworlds domain consist of a set of named blocks that can be stacked on top of one\nanother or placed on a table. The goal is to move the blocks from an initial configuration\nto a target configuration using a sequence of valid actions, namely:pick-up, stack,\nput-down, unstack. Any action can only be performed after required pre-conditions\nhave been met, upon performing the action the relevant state vari

In [55]:
#from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

template = '''Answer the question based on the given context. Incase, you 
cannot answer the question based SOLELY on the context, reply "I dont know".

Context: {context}
Question: {question}

'''
prompt = PromptTemplate(template=template, input_variables=['context','question'])

print(prompt.format(
    context="Hi I am susan.",
    question="What is my name?"
))

Answer the question based on the given context. Incase, you 
cannot answer the question based SOLELY on the context, reply "I dont know".

Context: Hi I am susan.
Question: What is my name?




In [56]:
chain = prompt | llm | parser
print(chain.invoke({
    "context":"Hi I am susan.",
    "question": "Was ist meine name?"
}))

<think>
Okay, so I'm trying to figure out what Susan's name is from the given context. Let me read through the text carefully.

The user starts with "Hi I am susan." That sounds like their first name, and it seems like they're using a contraction for "susanne," which means susan in French. So Susanne would be her full name if she's French, but I don't think that's the case here because the question is about her actual name.

Wait, but sometimes people just use their first name without the middle names when it's not needed or assumed. If that's the case, then Susan might be referring to herself as "susanne," which is a common contraction for her full name in French-speaking countries like France, Germany, or Italy. So if she says "Hi I am susan," it could mean she's saying hello and her first name is susanne.

But I'm not sure about that. Maybe the question is simpler. It asks "Was ist meine name?" which means "What is my name?" in German. That would be a direct question to confirm thei

### step 2. Grading top retrieved document ###

In [113]:
# LLM for Grading: grades the top document fetched by the retreiver 
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an assistant that evaluates documents for relevance and outputs JSON."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Give a binary 'yes' or 'no' to grade whether the document is relevant to the question. \n
    Provide the single binary grade as a JSON with a single key 'grade' and no premable or explanation.
    Document: {document}
    Question: {question}
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

#dummy run
inputs = {'document': [''], 'question':'This is my question'}
formatted_messages = chat_prompt.format_messages(**inputs)
print('Prompt to retrieval grader')
for msg in formatted_messages:
    print(f"{msg.type} : {msg.content}")

Prompt to retrieval grader
system : You are an assistant that evaluates documents for relevance and outputs JSON.
human : Give a binary 'yes' or 'no' to grade whether the document is relevant to the question. 

    Provide the single binary grade as a JSON with a single key 'grade' and no premable or explanation.
    Document: ['']
    Question: This is my question



In [120]:
retrieval_grader = chat_prompt | llm | JsonOutputParser()
#retrieval_grader = chat_prompt | llm 

In [122]:
#dummy run
question = "What is the motivation behind this project?"
docs = retriever.invoke(question)
doc_text = docs[0].page_content

grade = retrieval_grader.invoke({"question": question, "document": doc_text})


#### Sidebar: processing llm output before parsing for robustness ####

In [ ]:
import regex as re
def fetch_json(text):
    match=re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        return match.group(0)
    return None

In [ ]:
raw_text=retrieval_grader.invoke(inputs)
clean_text=fetch_json(raw_text)
json_parser = JsonOutputParser()
if clean_text is not None:
    grade=json_parser.parse(clean_text)
print(grade)

In [117]:
print(raw_text)

<think>
Okay, so I need to figure out whether the document is relevant to the question about the motivation behind a project. The question is asking for a 'yes' or 'no' response with a JSON object containing the key 'grade'. 

Looking at the document provided, it's labeled as "List of Figures" and has two sections. Section 14 talks about figures illustrating examples in Blocksworld, discussing actions and their validity. Section 15 compares actions to a simulated game using a distance metric. Both sections are about how distances progress towards the goal.

The question is about motivation, which is about the purpose or reasoning behind the project's goals. The document discusses how valid and invalid actions are explored, model responses, and metrics for progression. These topics don't directly address why the project started or what motivates it beyond its structure and objectives.

So, the document doesn't provide any information that would answer the question about motivation. It f

#### END sidebar: processing llm output before parsing for robustness ####

In [124]:
grade['grade']

'no'

In [ ]:
print(f"Retreiver fetched document: {doc_text}")

### step 3: Generate RAG response chain    ###

In [125]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an assistant for question-answering tasks."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Use the following context to answer the question in three sentences max. If you don't know the answer, just say so. 
Limit your response to a maximum of three sentences.
    Question: {question}
    Context: {context}                                                 
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

rag_chain = prompt | llm | StrOutputParser()


In [128]:
#dummy run
docs = retriever.invoke(question)
content_docs = " ".join([doc.page_content for doc in docs])
generated_response = rag_chain.invoke({"context": content_docs, "question": question})

<think>
Okay, so I need to figure out the motivation behind this project. Let me start by looking at the context provided.

The context includes several sections from a document, each starting with numbers 14 and 52, followed by definitions or key points. The first part talks about comparing configurations in Blocksworld, showing valid vs invalid actions, and using distance metrics to understand model responses. It mentions that without considering the initial state's top and bottom, certain actions are seen as invalid. This suggests they're trying to validate or refine how these models operate.

Then there are figures labeled 10 through 23. Figure 10 starts with an example problem instance in the Blocksworld domain, followed by a plan being acted upon. Figures 11-17 show various configurations and examples, especially comparing initial and goal states. Figure 18 introduces a natural language representation of these states. Figures 19-23 detail the prompt template for generating plans,

In [ ]:
print(generated_response)

In [ ]:
content_docs

### step 4: Check for Hallucinations ###

In [137]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are a grader assessing whether an answer is supported by a set of facts and outputs JSON."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Give a binary 'yes' or 'no' to grade whether the answer is supported by the given set of facts. \n
    Provide the single binary grade as a JSON with a single key 'grade' and no premable or explanation.
    
    Answer: {generation}
    Documents: {documents}
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
hallucination_grader = chat_prompt | llm | JsonOutputParser()

In [138]:
#dummy run
grade = hallucination_grader.invoke({"documents": content_docs, "generation": generated_response})

{'grade': 'yes'}


In [139]:
print(grade)

{'grade': 'yes'}


### step 5: set up Google search API ###

In [141]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
# from pydantic import BaseModel, Field, root_validator

search = GoogleSearchAPIWrapper(k=3,
                                google_api_key=GOOGLE_SEARCH_API_KEY,
                                google_cse_id=GOOGLE_CSE_ID)
def search_results(query):
    return search.results(query, num_results=3)

web_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search_results
)

In [142]:
#dummy run
question = "What is the motivation behind this project?"
results=web_search_tool.invoke({'query':question})
results

[{'title': 'Lack Of Motivation And Enthusiasm: How To Cope | BetterHelp',
  'link': 'https://www.google.com/share.google?q=PrBUcpiMIPxj4ItVs',
  'snippet': 'It can lead to a lack of achievement motivation, as individuals may not have the energy or drive to tackle new tasks or projects. Burnout may occur in any area\xa0...'},
 {'title': 'ADHD Paralysis Is Real: Here Are 8 Ways to Overcome It',
  'link': 'https://www.google.com/share.google?q=3A5mBgs8BhHJe3NCD',
  'snippet': "Every item counts toward completing the bigger project—even if it's an easy win! Every item you complete helps build motivation and foster a sense of\xa0..."},
 {'title': "'The Project' explores Project 2025's origins and goals to reshape ...",
  'link': 'https://www.google.com/share.google?q=pD04zZcJnK5fww6tL',
  'snippet': 'May 1, 2025 ... David Graham\'s new book, "The Project," details the origins of Project 2025 and its sweeping goals to reshape American culture.'}]

#### neccesarry declarations before langgraph ####

In [32]:
from typing_extensions import TypedDict
from typing import List
from langchain_core.documents import Document

In [ ]:
#state
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes: 
    question: string of the question
    generation: LLM generation 
    web_search: whether to add search 
    documents: list of retreived documents from vector store
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

#node
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args: 
        state (dict) : Current graph state

    Returns: 
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    print("----RETRIEVE----")
    question = state['question']

    #retrieval
    documents = retriever.invoke(question)
    return {'documents':documents, 'question':question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("----GENERATE----")

    question=state['question']
    documents=state['documents']

    #RAG generation 
    generation = rag_chain.invoke({'context':documents,'question':question})
    return {'documents':documents,
            'question':question,
            'generation':generation}

def grade_documents(state):
    
    """
    If any document is not relevant, we set a flag to run web search

    Determines whether the retrieved documents are relevant to the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs=[]
    web_search_flag='No'

    for d in documents:
        score = retrieval_grader.invoke(
            {'question':question,'document':d.page_content}
        )

        grade = score['score']
        
        #document is relevant

        if grade.lower() == 'yes':
            print("--GRADE: DOCUMENT RELEVANT--")
            filtered_docs.append(d)
        else:
            print("--GRADE: DOCUMENT NOT!!!! RELEVANT--")
            web_search_flag='Yes'
            continue
    
    return {'documents':filtered_docs,
            'question':question,
            'web_search_flag':web_search_flag}

def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("----WEB SEARCH----")
    question = state["question"]
    documents = state["documents"]

    # Web search
    #to-do
    docs = search.invoke({"query": question})
    # web_results = "\n".join([d["content"] for d in docs])
    web_results = "\n".join([d["snippet"] for d in docs])

    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

# Conditional i.e "if/else" edge
def route_question(state):
    """
    Route question to either web search or RAG. 
    Args: 
        state (dict): The current graph state
    Returns: 
        str: Next node to call
    """
    print("---ROUTE USER QUESTION---")
    question = state['question']
    print(question)
    source = question_router.invoke({"question":question})
    print(source)
    print(source["datasource"])
    if source['datasource'] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"  



            

### step 4 : create a langgraph ###

In [31]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae



NameError: name 'GraphState' is not defined

#### installations ####

In [46]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
